- https://scikit-learn.org/stable/modules/tree.html
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html

In [262]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 6)
plt.rcParams['font.size'] = 14
import pandas as pd
import sklearn as sk

In [263]:
df = pd.read_csv('../data/adult.data', index_col=False)

In [264]:
golden = pd.read_csv('../data/adult.test', index_col=False)

In [265]:
golden.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [266]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [267]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [268]:
from sklearn import preprocessing

In [269]:
enc = preprocessing.OrdinalEncoder()

In [270]:
transform_columns = ['sex']
non_num_columns = ['workclass', 'education', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'native-country']

In [271]:
pd.get_dummies(df[transform_columns]).head()

,sex_ Female,sex_ Male
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [272]:
x = df.copy()

x = pd.concat([x.drop(non_num_columns, axis=1), 
               pd.get_dummies(df[transform_columns])], axis=1,)

x["salary"] = enc.fit_transform(df[["salary"]])

In [273]:
x.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,sex_ Female,sex_ Male
0,39,77516,13,2174,0,40,0.0,0,1
1,50,83311,13,0,0,13,0.0,0,1
2,38,215646,9,0,0,40,0.0,0,1
3,53,234721,7,0,0,40,0.0,0,1
4,28,338409,13,0,0,40,0.0,1,0


In [274]:
xt = golden.copy()

xt = pd.concat([xt.drop(non_num_columns, axis=1), 
               pd.get_dummies(golden[transform_columns])], axis=1,)

xt["salary"] = enc.fit_transform(golden[["salary"]])

In [275]:
xt.salary.value_counts()

0.0    12435
1.0     3846
Name: salary, dtype: int64

In [276]:
enc.categories_

[array([' <=50K.', ' >50K.'], dtype=object)]

In [277]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#### Choose the model of your preference: DecisionTree or RandomForest

In [278]:
model = RandomForestClassifier(criterion='entropy')

In [279]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=None)

In [280]:
model.fit(x.drop(['fnlwgt','salary'], axis=1), x.salary)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [281]:
model.tree_.node_count

8337

In [282]:
list(zip(x.drop(['fnlwgt','salary'], axis=1).columns, model.feature_importances_))

[('age', 0.3229956250663931),
 ('education-num', 0.1599807712183501),
 ('capital-gain', 0.22710347533365707),
 ('capital-loss', 0.07898524108569598),
 ('hours-per-week', 0.15521122568227538),
 ('sex_ Female', 0.020999499657367266),
 ('sex_ Male', 0.03472416195626108)]

In [283]:
list(zip(x.drop(['fnlwgt','salary'], axis=1).columns, model.feature_importances_))

[('age', 0.3229956250663931),
 ('education-num', 0.1599807712183501),
 ('capital-gain', 0.22710347533365707),
 ('capital-loss', 0.07898524108569598),
 ('hours-per-week', 0.15521122568227538),
 ('sex_ Female', 0.020999499657367266),
 ('sex_ Male', 0.03472416195626108)]

In [284]:
x.drop(['fnlwgt','salary'], axis=1).head()

,age,education-num,capital-gain,capital-loss,hours-per-week,sex_ Female,sex_ Male
0,39,13,2174,0,40,0,1
1,50,13,0,0,13,0,1
2,38,9,0,0,40,0,1
3,53,7,0,0,40,0,1
4,28,13,0,0,40,1,0


In [285]:
set(x.columns) - set(xt.columns)

set()

In [286]:
list(x.drop('salary', axis=1).columns)

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'sex_ Female',
 'sex_ Male']

In [287]:
predictions = model.predict(xt.drop(['fnlwgt','salary'], axis=1))
predictionsx = model.predict(x.drop(['fnlwgt','salary'], axis=1))

In [288]:
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix, auc, roc_curve
)

In [289]:
accuracy_score(xt.salary, predictions)

0.8206498372335852

In [290]:
accuracy_score(xt.salary, predictions)

0.8206498372335852

In [291]:
confusion_matrix(xt.salary, predictions)

array([[11459,   976],
       [ 1944,  1902]], dtype=int64)

In [292]:
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.85      0.92      0.89     12435
         1.0       0.66      0.49      0.57      3846

    accuracy                           0.82     16281
   macro avg       0.76      0.71      0.73     16281
weighted avg       0.81      0.82      0.81     16281



In [293]:
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.85      0.92      0.89     12435
         1.0       0.66      0.49      0.57      3846

    accuracy                           0.82     16281
   macro avg       0.76      0.71      0.73     16281
weighted avg       0.81      0.82      0.81     16281



In [294]:
accuracy_score(x.salary, predictionsx)

0.8955806025613464

In [295]:
confusion_matrix(x.salary, predictionsx)

array([[24097,   623],
       [ 2777,  5064]], dtype=int64)

In [296]:
print(classification_report(x.salary, predictionsx))

              precision    recall  f1-score   support

         0.0       0.90      0.97      0.93     24720
         1.0       0.89      0.65      0.75      7841

    accuracy                           0.90     32561
   macro avg       0.89      0.81      0.84     32561
weighted avg       0.90      0.90      0.89     32561



In [297]:
print(classification_report(x.salary, predictionsx))

              precision    recall  f1-score   support

         0.0       0.90      0.97      0.93     24720
         1.0       0.89      0.65      0.75      7841

    accuracy                           0.90     32561
   macro avg       0.89      0.81      0.84     32561
weighted avg       0.90      0.90      0.89     32561



# For the following use the above `adult` dataset. Start with only numerical features/columns.  

# 1. Show the RandomForest outperforms the DecisionTree for a fixed `max_depth` by training using the train set and `precision`, `recall`, `f1` on golden-test set.

In [298]:
model_tree = DecisionTreeClassifier(max_depth = 5)
model_forest = RandomForestClassifier(max_depth = 5)

In [299]:
x = df.copy()

x = pd.concat([x.drop(non_num_columns, axis=1), 
               pd.get_dummies(df[transform_columns])], axis=1,)

x["salary"] = enc.fit_transform(df[["salary"]])

In [300]:
xt = golden.copy()

xt = pd.concat([xt.drop(non_num_columns, axis=1), 
               pd.get_dummies(golden[transform_columns])], axis=1,)

xt["salary"] = enc.fit_transform(golden[["salary"]])

In [301]:
num_columns = ['salary','age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
x = x[num_columns]
xt = xt[num_columns]

In [302]:
model_tree.fit(x.drop(['salary'], axis = 1), x.salary)

predictions = model_tree.predict(xt.drop(['salary'], axis = 1))

### Tree Model Metrics with Depth of 5

In [303]:
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90     12435
         1.0       0.76      0.42      0.54      3846

    accuracy                           0.83     16281
   macro avg       0.80      0.69      0.72     16281
weighted avg       0.82      0.83      0.81     16281



### Forest Model Metrics with Depth of 5

In [304]:
model_forest.fit(x.drop(['salary'], axis = 1), x.salary)

predictions = model_forest.predict(xt.drop(['salary'], axis = 1))
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90     12435
         1.0       0.77      0.41      0.53      3846

    accuracy                           0.83     16281
   macro avg       0.80      0.69      0.72     16281
weighted avg       0.82      0.83      0.81     16281



C:\Users\Ben\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# 2. For RandomForest or DecisionTree and using the `adult` dataset, systematically add new columns, one by one, that are non-numerical but converted using the feature-extraction techniques we learned. Show [`precision`, `recall`, `f1`] for each additional feature added.

In [305]:
import sklearn.preprocessing as preprocessing

In [306]:
x = df.copy()

x = pd.concat([x.drop(non_num_columns, axis=1), 
               pd.get_dummies(df[transform_columns])], axis=1,)

x["salary"] = enc.fit_transform(df[["salary"]])

In [307]:
xt = golden.copy()

xt = pd.concat([xt.drop(non_num_columns, axis=1), 
               pd.get_dummies(golden[transform_columns])], axis=1,)

xt["salary"] = enc.fit_transform(golden[["salary"]])

In [308]:
num_columns = ['salary','age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
x = x[num_columns]
xt = xt[num_columns]

In [309]:
to_add = set(df.columns) - set(x.columns)
to_add.remove('fnlwgt')
to_add.remove('education')
to_add.remove('relationship')

### Model With Marital Status

In [310]:
x

,salary,age,education-num,capital-gain,capital-loss,hours-per-week
0,0.0,39,13,2174,0,40
1,0.0,50,13,0,0,13
2,0.0,38,9,0,0,40
3,0.0,53,7,0,0,40
4,0.0,28,13,0,0,40
...,...,...,...,...,...,...
32556,0.0,27,12,0,0,38
32557,1.0,40,9,0,0,40
32558,0.0,58,9,0,0,40
32559,0.0,22,9,0,0,20


In [311]:
enc = preprocessing.OrdinalEncoder()
enc.fit(df['marital-status'].values.reshape(-1,1))

x['marital-status'] = enc.transform(df['marital-status'].values.reshape(-1,1))
xt['marital-status'] = enc.transform(golden['marital-status'].values.reshape(-1,1))

In [312]:
model_forest.fit(x.drop(['salary'], axis = 1), x.salary)


predictions = model_forest.predict(xt.drop(['salary'], axis = 1))
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.87      0.95      0.91     12435
         1.0       0.78      0.53      0.63      3846

    accuracy                           0.85     16281
   macro avg       0.82      0.74      0.77     16281
weighted avg       0.85      0.85      0.84     16281



### Model With Marital Status, Native Country

In [313]:
enc = preprocessing.OrdinalEncoder()
enc.fit(df['native-country'].values.reshape(-1,1))

x['native-country'] = enc.transform(df['native-country'].values.reshape(-1,1))
xt['native-country'] = enc.transform(golden['native-country'].values.reshape(-1,1))

In [314]:
model_forest.fit(x.drop(['salary'], axis = 1), x.salary)

predictions = model_forest.predict(xt.drop(['salary'], axis = 1))
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.86      0.96      0.91     12435
         1.0       0.80      0.51      0.62      3846

    accuracy                           0.85     16281
   macro avg       0.83      0.74      0.77     16281
weighted avg       0.85      0.85      0.84     16281



### Model With Marital Status, Native Country, Occupation

In [315]:
enc = preprocessing.OrdinalEncoder()
enc.fit(df['occupation'].values.reshape(-1,1))

x['occupation'] = enc.transform(df['occupation'].values.reshape(-1,1))
xt['occupation'] = enc.transform(golden['occupation'].values.reshape(-1,1))

In [316]:
model_forest.fit(x.drop(['salary'], axis = 1), x.salary)

predictions = model_forest.predict(xt.drop(['salary'], axis = 1))
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.86      0.96      0.91     12435
         1.0       0.79      0.49      0.61      3846

    accuracy                           0.85     16281
   macro avg       0.82      0.73      0.76     16281
weighted avg       0.84      0.85      0.84     16281



### Model With Marital Status, Native Country, Occupation, Sex

In [317]:
enc = preprocessing.OrdinalEncoder()
enc.fit(df['sex'].values.reshape(-1,1))

x['sex'] = enc.transform(df['sex'].values.reshape(-1,1))
xt['sex'] = enc.transform(golden['sex'].values.reshape(-1,1))

In [318]:
model_forest.fit(x.drop(['salary'], axis = 1), x.salary)

predictions = model_forest.predict(xt.drop(['salary'], axis = 1))
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.85      0.97      0.91     12435
         1.0       0.84      0.45      0.58      3846

    accuracy                           0.85     16281
   macro avg       0.85      0.71      0.75     16281
weighted avg       0.85      0.85      0.83     16281



### Model With Marital Status, Native Country, Occupation, Sex, Workclass

In [319]:
enc = preprocessing.OrdinalEncoder()
enc.fit(df['workclass'].values.reshape(-1,1))

x['workclass'] = enc.transform(df['workclass'].values.reshape(-1,1))
xt['workclass'] = enc.transform(golden['workclass'].values.reshape(-1,1))

In [320]:
model_forest.fit(x.drop(['salary'], axis = 1), x.salary)

predictions = model_forest.predict(xt.drop(['salary'], axis = 1))
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.85      0.97      0.91     12435
         1.0       0.83      0.45      0.58      3846

    accuracy                           0.85     16281
   macro avg       0.84      0.71      0.74     16281
weighted avg       0.85      0.85      0.83     16281



# 3. Optional: Using gridSearch find the most optimal parameters for your model
Warning: this can be computationally intensive and may take some time.
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://scikit-learn.org/stable/modules/grid_search.html